In [45]:
import pandas as pd
from raceplotly.plots import barplot

In [53]:
athlete_Event_Results = pd.read_csv('Dataset/Olympic_Athlete_Event_Results.csv', sep=',')
athlete_Event_Results = athlete_Event_Results.replace(["na"], None)


athlete_results = athlete_Event_Results[athlete_Event_Results["edition"].str.contains("Winter") == False]   # apenas jogos de verao

athlete_results = athlete_results.reset_index()

athlete_results = athlete_results.drop(columns=['result_id','athlete','athlete_id','pos'])


repeat = []
i=0
while(i < len(athlete_results)):
    if athlete_results.loc[i, 'isTeamSport'] == True:
        auxi = i + 1 
        p = athlete_results.loc[i, 'country_noc']
        e = athlete_results.loc[i, 'event']
        a = athlete_results.loc[i, 'edition_id']
        while((athlete_results.loc[auxi, 'country_noc'] == p) and (athlete_results.loc[auxi, 'event'] == e) and (athlete_results.loc[auxi, 'edition_id'] == a)):
            repeat.append(auxi)
            auxi += 1
        i = auxi-1
    i+=1

athlete_results.drop(repeat, inplace=True)
athlete_results.drop(columns=['edition_id', 'isTeamSport', 'index'], inplace=True)
athlete_results['edition'] = athlete_results['edition'].str.split().str[0].astype(int)

athlete_results["medal"] = athlete_results["medal"].notnull().mul(1)

display(athlete_results)

,edition,country_noc,sport,event,medal
0,1908,ANZ,Athletics,"100 metres, Men",0
1,1908,ANZ,Athletics,"400 metres, Men",0
2,1908,ANZ,Athletics,"800 metres, Men",0
3,1908,ANZ,Athletics,"800 metres, Men",0
4,1908,ANZ,Athletics,"800 metres, Men",0
...,...,...,...,...,...
253177,2020,USA,Wrestling,"Featherweight, Freestyle, Women",0
253178,2020,USA,Wrestling,"Lightweight, Freestyle, Women",1
253179,2020,USA,Wrestling,"Middleweight, Freestyle, Women",0
253180,2020,USA,Wrestling,"Light-Heavyweight, Freestyle, Women",1


In [55]:
sport = "Athletics"

athlete_results = athlete_results[athlete_Event_Results['sport'] == sport]

df = pd.DataFrame(athlete_results.groupby(['edition','country_noc']).sum()).reset_index()

df

C:\Users\emanu\AppData\Local\Temp\ipykernel_19976\4276111948.py:3: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,edition,country_noc,medal
0,1896,BUL,0
1,1896,FRA,0
2,1896,GRE,12
3,1900,BOH,1
4,1900,CUB,2
...,...,...,...
1266,2020,USA,20
1267,2020,UZB,2
1268,2020,VEN,0
1269,2020,VIE,0


In [56]:
new_medal = []
for i in range(len(df)):
    edition = df.loc[i, 'edition']
    country = df.loc[i, 'country_noc']
    medals = df.loc[i, 'medal']

    new_medal.append(df[(df['edition'] <= edition) & (df['country_noc'] == country)]['medal'].sum())

df['medals'] = new_medal
df.drop(columns=['medal'], inplace=True)

df

,edition,country_noc,medals
0,1896,BUL,0
1,1896,FRA,0
2,1896,GRE,12
3,1900,BOH,1
4,1900,CUB,2
...,...,...,...
1266,2020,USA,323
1267,2020,UZB,8
1268,2020,VEN,2
1269,2020,VIE,2


In [67]:

#1 Assign color to each unique item
from random import sample
import plotly.graph_objects as go

colors = {item: 'rgb({}, {}, {})'.format(*sample(range(256), 3)) for item in df['country_noc'].unique()}
df['color'] = df['country_noc'].map(colors)

my_raceplot = barplot(df,  item_column='country_noc', value_column='medals', time_column='edition')

my_raceplot.plot(item_label = 'Top 10 crops', value_label = 'Production quantity (tonnes)', frame_duration = 800)


In [68]:
data = pd.read_csv('https://raw.githubusercontent.com/lc5415/raceplotly/main/example/dataset/FAOSTAT_data.csv')

display(data)

my_raceplot = barplot(data,  item_column='Item', value_column='Value', time_column='Year')

my_raceplot.plot(item_label = 'Top 10 crops', value_label = 'Production quantity (tonnes)', frame_duration = 800)

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,QC,Crops,5000,World,5510,Production,800,Agave fibres nes,1961,1961,tonnes,34600.0,A,"Aggregate, may include official, semi-official..."
1,QC,Crops,5000,World,5510,Production,800,Agave fibres nes,1962,1962,tonnes,32950.0,A,"Aggregate, may include official, semi-official..."
2,QC,Crops,5000,World,5510,Production,800,Agave fibres nes,1963,1963,tonnes,35000.0,A,"Aggregate, may include official, semi-official..."
3,QC,Crops,5000,World,5510,Production,800,Agave fibres nes,1964,1964,tonnes,35124.0,A,"Aggregate, may include official, semi-official..."
4,QC,Crops,5000,World,5510,Production,800,Agave fibres nes,1965,1965,tonnes,35721.0,A,"Aggregate, may include official, semi-official..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9390,QC,Crops,5000,World,5510,Production,135,Yautia (cocoyam),2014,2014,tonnes,548968.0,A,"Aggregate, may include official, semi-official..."
9391,QC,Crops,5000,World,5510,Production,135,Yautia (cocoyam),2015,2015,tonnes,515236.0,A,"Aggregate, may include official, semi-official..."
9392,QC,Crops,5000,World,5510,Production,135,Yautia (cocoyam),2016,2016,tonnes,485076.0,A,"Aggregate, may include official, semi-official..."
9393,QC,Crops,5000,World,5510,Production,135,Yautia (cocoyam),2017,2017,tonnes,442831.0,A,"Aggregate, may include official, semi-official..."
